## Description:
> 这个情感分类项目是研一上学期选的数据挖掘项目。 此次情感分类项目的任务是给定训练集（句子和情感标签），根据所学知识训练出一个模型，然后给定测试集，测试集的每一个样例是一句话，根据模型推断所属的情感。 <br>
> 与一般的情感分类的不同之处： 
>> * 这不是一个二分类的任务，是一个6分类的任务。
>> * 这不是一个简单的单分类任务，而也是一个多分类任务，即一句话可能对应多个标签
> 
> 任务分析：
>> 基于上面的要求，这次用深度学习的模型进行尝试，做一个基于深度学习的情感分类。因为机器学习的模型的效果比较差，可能对于词向量来说，获取无法体现出句子先后的逻辑特征，所以这里采用目前最先进的自然语言处理模型-BERT，参考github(BERT_Chinese_Classification-master)，采用谷歌提供的训练好的BERT模型的接口，然后fine-tune。关于BERT的原理，可以参考GitHub(BERT_tutorial)。B站上也有视频讲的挺好的。<br>
任务分为下面几部分进行：
>> * 关于BERT接口的使用，见下面的描述。这里关键是要进行数据集的准备，因为接口中的数据集要求的格式是.txt或者是.csv， 并且需要经过一些处理，所以这里主要是数据集的准备。
>> * 关于数据集准备，训练集和测试集都需要是.txt文件，所以进行处理，并且由于目前测试集还没出来，所以在总的数据集里面分出10%作为测试集。
>> * 划分完数据集之后，剩下的就交给BERT了，采用的环境是谷歌的colaboratory，GPU模式。
>> * 预测完之后，结果是通过softmax预测的每个类的概率，所以，还需要转成最后提交形式的结果，这块也需要自己处理，由于可能是一个样本对应了多个标签，所以进行最终结果选择的时候，要根据概率进行筛选，如果相差不大的，就可以都写上。
>
>PS： 发现.txt之后的数据集效果并不是很好，所以改成了.tsv数据集，参考的一篇[博客](https://blog.csdn.net/xavier_muse/article/details/95729133)，新写了一个processor。所以后面用这个吧。

In [1]:
"""导入用到的两个包"""
import pandas as pd
from sklearn.model_selection import train_test_split

## 处理数据集，还是和机器学习的处理一样

In [2]:
df = pd.read_excel('../Dataset/train.xlsx', index_col=False, names=['data','label'],encoding='utf-8')
df['data'] = df['data'].astype(str)

In [3]:
for d, l in df.iterrows():
    l.label = l.label.replace('[','').replace(']','').replace("'",'').split(',')

In [4]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()

In [5]:
for i in range(df.shape[0]):
    temp = df.iloc[i]
    if (len(temp['label'])>1):
        for item in temp['label']:
            df2 = df2.append({"data":temp['data'], "label":item.strip()}, ignore_index=True)
    else:
        df1 = df1.append(temp, ignore_index=True)

In [6]:
print(df1.shape)
print(df2.shape)

(5364, 2)
(1310, 2)


In [7]:
for i,v in df1['label'].items():
    if (type(v) == list):
        df1['label'][i] = v[0]

In [8]:
df = pd.concat([df1, df2], axis=0, ignore_index=True)

In [17]:
df.shape

(6674, 2)

In [18]:
df

,data,label
0,【 恶魔们 的 高跟鞋 】 gasoline glamour shoes 集锦 ~ ~ ！ ！ ！,None
1,经过 长途跋涉 ， 终于 到 了 威尼斯 。 晩上 从 机场 搭乘 boat taxl ， ...,Happiness
2,hello kitty 」 停 不 了 的 爱 。,Happiness
3,唱 错词 了 哈哈 好 可爱 坑 死 日本人 - 评论 视频 ： 1991 年 【 beyo...,Happiness
4,我 是 能量 满 分 选手 ， 我们 又 聊到 这么 晚 才 回家 ， 但是 如果 还 有 ...,Happiness
5,kelly clarkson长 得 太 肥 了 ， 她 照 镜子 的 时候 应该 在 安慰 ...,None
6,在 看 《 绯闻 女孩 第五 季 》 ： 是 不 是 太 久 没 看 美剧 了 ， 连 go...,Surprise
7,当 我 看到 “ jane love ” 这个 名字 ， 当 我 看到 那 一 句 俗气 的...,Anger
8,真心 觉得 iu 的 每 首 歌 都 差 不 多 啊 ~ ~ 最近 大发 的 你 和 我 也...,None
9,年29 大家 都 团团 圆圆 吃 团年 饭 咯 ， 天气 骤降 ， 冷冷 的 才 有 新年 ...,Happiness


In [19]:
df['label'].value_counts()

None         1896
Happiness    1823
Sadness      1086
Surprise      651
Fear          648
Anger         570
Name: label, dtype: int64

In [20]:
label_mapping = {"None": 1, "Happiness": 2, "Sadness": 3, "Surprise": 4, "Fear": 5, "Anger": 6}
df['label'] = df['label'].map(label_mapping)

In [21]:
df.shape

(6674, 2)

## 划分数据集，并且写入.txt文件
> 由于目前没有测试集，所以需要这样做，如果后面测试集来了之后，就不用划分了，只需要处理完转成.txt即可。后面我会把这个数据处理的这个写成一个函数。

In [29]:
x = df['data']
y = df['label']

In [30]:
"""划分数据集"""
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.1, random_state=8)

In [31]:
traintxt = pd.DataFrame({"data":X_train, "label":Y_train})
testtxt = pd.DataFrame({"data": X_test, "label": Y_test})

In [32]:
"""索引重置"""
traintxt.reset_index(inplace=True)
traintxt.drop('index', axis=1, inplace=True)

testtxt.reset_index(inplace=True)
testtxt.drop('index', axis=1, inplace=True)

In [33]:
traintxt.to_csv("BERT/data/train.txt", sep='\t')
testtxt.to_csv("BERT/data/test.txt", sep='\t')

"""这样就写入到BERT的数据集里面去了，但是还有个小问题，就是需要找到那两个.txt，打开，把首行的标签给去掉，接下来就是BERT的任务了，转入
colaboratory进行BERT模型的微调和产生结果了。具体操作，把两个.txt上传到谷歌云盘的data文件夹，然后就是运行runclassifier.py文件即可"""

'这样就写入到BERT的数据集里面去了，但是还有个小问题，就是需要找到那两个.txt，打开，把首行的标签给去掉，接下来就是BERT的任务了，转入\ncolaboratory进行BERT模型的微调和产生结果了。具体操作，把两个.txt上传到谷歌云盘的data文件夹，然后就是运行runclassifier.py文件即可'

## 数据集准备写成一个函数
> 这里我把上面的数据集的准备过程写成一个函数，因为目前没有测试集，才写成上面这个样子，如果测试集有了之后，就可以直接训练集和测试集分开处理写入.txt即可。 所以测试集下来之后，上面的这个过程统一使用下面这个函数进行。

In [41]:
import os

In [42]:
def DataProcess(filename):
    path = '../Dataset/'
    filepath = os.path.join(path, filename)
    
    # 测试集需要先打开，把标签那一列设置成全0
    df = pd.read_excel(filepath, index_col=False, names=['data','label'],encoding='utf-8')
    df['data'] = df['data'].astype(str)
    
    # 训练集和测试集得区分开对待, 测试集本身是对的，直接存入即可
    if filename=='train.xlsx':
        for d, l in df.iterrows():
            l.label = l.label.replace('[','').replace(']','').replace("'",'').split(',')
            
        df1 = pd.DataFrame()
        df2 = pd.DataFrame()
        
        for i in range(df.shape[0]):
            temp = df.iloc[i]
            if (len(temp['label'])>1):
                for item in temp['label']:
                    df2 = df2.append({"data":temp['data'], "label":item.strip()}, ignore_index=True)
            else:
                df1 = df1.append(temp, ignore_index=True)

        
        for i,v in df1['label'].items():
            if (type(v) == list):
                df1['label'][i] = v[0]
        
        df = pd.concat([df1, df2], axis=0, ignore_index=True)
        
        label_mapping = {"None": 1, "Happiness": 2, "Sadness": 3, "Surprise": 4, "Fear": 5, "Anger": 6}
        df['label'] = df['label'].map(label_mapping)
        df.to_csv("BERT/data/train1.txt", sep='\t')
        
    else: 
        df.to_csv("BERT/data/test.txt", sep='\t')

In [43]:
"""调用函数进行数据集准备"""
DataProcess('train.xlsx')
#DataProcess('test.xlsx')

## 准备.tsv数据集
> 我发现.txt数据集不太稳定， 所以根据一个[博客](https://blog.csdn.net/xavier_muse/article/details/95729133)，准备了.tsv数据集，原标签也没有变。
>.tsv数据集类似.csv数据集，只不过用tab进行分割
> 这里调换一下顺序，读入原训练集文件之后，第一列是label，第二列是句子，然后写入.tsv文件中

In [1]:
"""导入用到的两个包"""
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_excel('../Dataset/train.xlsx', index_col=False, names=['data','label'],encoding='utf-8')
df['data'] = df['data'].astype(str)

In [11]:
for d, l in df.iterrows():
     l.label = l.label.replace('[','').replace(']','').replace("'",'').split(',')
            
df1 = pd.DataFrame()
df2 = pd.DataFrame()
        
for i in range(df.shape[0]):
    temp = df.iloc[i]
    if (len(temp['label'])>1):
         for item in temp['label']:
            df2 = df2.append({"data":temp['data'], "label":item.strip()}, ignore_index=True)
    else:
            df1 = df1.append(temp, ignore_index=True)

        
for i,v in df1['label'].items():
     if (type(v) == list):
        df1['label'][i] = v[0]
        
df = pd.concat([df1, df2], axis=0, ignore_index=True)

In [12]:
df

,data,label
0,【 恶魔们 的 高跟鞋 】 gasoline glamour shoes 集锦 ~ ~ ！ ！ ！,None
1,经过 长途跋涉 ， 终于 到 了 威尼斯 。 晩上 从 机场 搭乘 boat taxl ， ...,Happiness
2,hello kitty 」 停 不 了 的 爱 。,Happiness
3,唱 错词 了 哈哈 好 可爱 坑 死 日本人 - 评论 视频 ： 1991 年 【 beyo...,Happiness
4,我 是 能量 满 分 选手 ， 我们 又 聊到 这么 晚 才 回家 ， 但是 如果 还 有 ...,Happiness
5,kelly clarkson长 得 太 肥 了 ， 她 照 镜子 的 时候 应该 在 安慰 ...,None
6,在 看 《 绯闻 女孩 第五 季 》 ： 是 不 是 太 久 没 看 美剧 了 ， 连 go...,Surprise
7,当 我 看到 “ jane love ” 这个 名字 ， 当 我 看到 那 一 句 俗气 的...,Anger
8,真心 觉得 iu 的 每 首 歌 都 差 不 多 啊 ~ ~ 最近 大发 的 你 和 我 也...,None
9,年29 大家 都 团团 圆圆 吃 团年 饭 咯 ， 天气 骤降 ， 冷冷 的 才 有 新年 ...,Happiness


In [13]:
label = df['label']
df.drop(['label'], axis=1, inplace=True)
df.insert(0, 'label', label)

In [14]:
df = shuffle(df, random_state=8)

In [15]:
df

,label,data
132,Happiness,爸爸 又 在 看 潮剧 了 ， 每 年 的 初一 总 免不了 跟着 听上 一 段 潮剧 。 ...
368,None,长假 过 得 差不多 了 ， 明天 大多数 人 就 要 正常 的 上班 生活 ， 发开 门利...
5675,Surprise,2012年 的 除夕 ， 我们 在 为 美国 的 eli 准备 生日 。 这边 的 生日 风...
6561,Surprise,回 老家 途中 无聊 直 到 把 i phone 玩到 没 电 还 没有 到 家 ， 突然 ...
1279,Happiness,早晨 好 ！ 一大早 收到 勇哥 的 喜帖 （ 婚礼 帮忙 的 礼金 ） ， 多 沾 点 喜...
4811,Happiness,《 yes or no 》 两 个 女孩 之间 的 爱情 ， 也 很 感人 ~
6082,Anger,为何 如此 失落 和 没 心情 ？ 明明 是 别人 什么 都 不 会 干 ， 我 干 嘛 要...
4861,Happiness,开开心心 过 龙年 、 、 丢掉 所有 的 烦心 事 、 、 happy new year ！ ~
238,None,"看看 人家 的 发言 就 知道 张13钗 为何 不 能 获奖 了 \ "" are truly..."
3903,None,最近 生活 的 主题 就 是 antisocial死宅 发霉 ， 想 把 自己 彻底 地 封...


In [16]:
"""6000多条数据，对于bert来说，感觉容易过拟合，所以这里给它扩充一下数据，复制3遍df，然后随机打乱"""
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
df5 = df.copy()

df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [17]:
df.shape

(33370, 2)

In [18]:
""" 再随机打乱"""
df = shuffle(df, random_state=8)

In [19]:
df.head()

,label,data
5323,Happiness,刚 看完 《 总统 杀局 》 the ides of march ， 乔治·克鲁尼 导演 和...
1819,Happiness,刘忻儿 ， 刘小鬼 ， 刘点点 对不起 ， 我们 迟到 了 七 年 真的 很 庆幸 ， 真的...
4516,None,当然 也 少 不 了 这 首 —— 《 seasons in the sun 》 。
3899,Happiness,喜欢 hebe 对 音乐 的 态度 ！ 她 的 演唱会 非常 精彩 《 田馥 甄 to my...
5489,Sadness,what ？ 好久 没 上 了 ， 前 一 阵 听说 吵架 了 …… 最近 竟然 挂掉 了 ...


In [21]:
x = df['data']
y = df['label']

In [26]:
"""划分数据集"""
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.1, random_state=8)

traintsv = pd.DataFrame({"label":y_train, "data":x_train})
devtsv = pd.DataFrame({"label":y_dev, "data": x_dev})

In [27]:
"""索引重置"""
traintsv.reset_index(inplace=True)
traintsv.drop('index', axis=1, inplace=True)

devtsv.reset_index(inplace=True)
devtsv.drop('index', axis=1, inplace=True)

In [28]:
traintsv.to_csv("BERT/data/train.tsv", sep='\t', index=False)
devtsv.to_csv("BERT/data/dev.tsv", sep='\t', index=False)

In [12]:
test = pd.read_excel("../Dataset/test.xlsx", header=None)
test.to_csv("BERT/data/test.tsv", sep='\t')

In [11]:
test

,0
0,我 收藏 了 ： why me-李宇 春
1,嗯 、 够钟 翻归 了 。 明天 顶住 个 翻工 。 —— at last 《 你 瞒 我 ...
2,《 鹿鼎记 ： 龙脉 传说 》 : 12月 23日 觉醒 公测 ， 追寻 龙脉 遗迹 ， 梦...
3,我 正在 收听 sarah connor 的 单曲 《 just one last danc...
4,呢 杯野 ！ 好＂勒 ＂ 啊 ！ blue mountain ！
5,马天宇 是 不 是 个 gay ？ 难道 没有 解答 ？
6,make a wish ! 克里斯多福 在 河南 老家 向weibo 全体 关注 我 的 亲...
7,肚子 好饿 肚子 好饿 ~ 啊 … 为什么 还 不 可以 吃饭 啊 为什么 为什么 最近 ...
8,听到 噼噼 啪啪 的 炮仗声 才 真正 感觉 到 过 年 啦 ！ 祝 大家 春节 快乐 心 ...
9,又 过 了 一 次 fate ， 留 在 记忆 中 的 ， 永远 是 那 最 完美 的 亚瑟...


##  结果生成
> 这一块是针对BERT对测试集返回的结果进行一个处理，生成最后要求上交的结果。BERT返回的结果是一个tsv文件，里面是每个测试样本对应的各种情感的概率，要做的就是选出概率比较大的几个作为这个样本的训练结果，比如，如果有大于90%的，那么就返回一个标签， 如果有两个差不多的，就返回两个标签等。然后做成老师要求的格式。

In [3]:
"""读取tsv文件，在BERT/result/result.tsv"""
res = pd.read_csv("./BERT/result/test_results.tsv", sep='\t', header=None)
res

,0,1,2,3,4,5
0,0.998559,0.000548,0.000053,0.000222,0.000500,0.000118
1,0.002124,0.001246,0.151825,0.001956,0.010963,0.831887
2,0.001600,0.997479,0.000285,0.000316,0.000252,0.000067
3,0.999673,0.000089,0.000060,0.000031,0.000057,0.000090
4,0.998968,0.000161,0.000080,0.000221,0.000250,0.000320
5,0.011080,0.000468,0.001434,0.978883,0.006011,0.002124
6,0.985087,0.013875,0.000090,0.000537,0.000225,0.000186
7,0.000295,0.000513,0.006910,0.000503,0.990181,0.001597
8,0.001099,0.927483,0.067734,0.000490,0.002670,0.000524
9,0.999633,0.000103,0.000085,0.000051,0.000056,0.000071


In [4]:
# 建立映射字典
label_mapping = {'1':"None", '2':"Happiness", '3':"Sadness", '4':"Surprise", '5':"Fear", '6':'Anger' }

In [5]:
"""生成结果, 由于测试集没到，所以这里先用index代替每个样本"""
# 遍历res的每一行的每一列，如果大于20%， 就算这个样本的结果
result = {}
res_array = res.values   # 转成二维矩阵
for i in range(res_array.shape[0]):
    label = []
    for j in range(res_array.shape[1]):
        if (res_array[i][j]>0.3):
            label.append(label_mapping[str(j+1)])
    result[str(i)] = label

In [6]:
list1 = []
list2 = []
for i in result.keys():
    list1.append(i)
    list2.append(result[i])

dictionary = {"id":list1, "labels": list2}

In [36]:
last_res = pd.DataFrame(dictionary)

In [37]:
"""测试集来了之后，把id这一列换成句子即可"""
last_res

,id,labels
0,0,[None]
1,1,[Anger]
2,2,[Happiness]
3,3,[None]
4,4,[None]
5,5,[Surprise]
6,6,[None]
7,7,[Fear]
8,8,[Happiness]
9,9,[None]


In [38]:
last_res = pd.concat([test[0], last_res['labels'].astype(str)], axis=1)

In [39]:
last_res.head()

,0,labels
0,我 收藏 了 ： why me-李宇 春,['None']
1,嗯 、 够钟 翻归 了 。 明天 顶住 个 翻工 。 —— at last 《 你 瞒 我 ...,['Anger']
2,《 鹿鼎记 ： 龙脉 传说 》 : 12月 23日 觉醒 公测 ， 追寻 龙脉 遗迹 ， 梦...,['Happiness']
3,我 正在 收听 sarah connor 的 单曲 《 just one last danc...,['None']
4,呢 杯野 ！ 好＂勒 ＂ 啊 ！ blue mountain ！,['None']


In [40]:
"""保存到文件"""
last_res.to_excel("../result/result.xlsx", index=False, header=None)

参考博客：https://blog.csdn.net/xavier_muse/article/details/95729133